# Camada raw
## Armazenando dados originais

Importando biblotecas necessárias

In [2]:
import pandas as pd

Lendo arquivo original em csv

In [18]:
rwzd_olist_order_reviews = pd.read_csv(
    filepath_or_buffer='../dataset/incoming/olist_order_reviews_dataset.csv',
    delimiter=',',
    quotechar='"',
    header=0,
    dtype={
        'review_id': str,
        'order_id': str,
        'review_score': pd.UInt8Dtype(),
        'review_comment_title': str,
        'review_comment_message': str,
        'review_creation_date': str,
        'review_answer_timestamp': str,
    },
    date_format='%Y-%m-%d %H:%M:%s',
    parse_dates=['review_creation_date', 'review_answer_timestamp'],
)

Assegurando tipo _datetime_ para colunas com datas

In [19]:
rwzd_olist_order_reviews['review_creation_date'] = rwzd_olist_order_reviews['review_creation_date'].astype('datetime64[ns]')
rwzd_olist_order_reviews['review_answer_timestamp'] = rwzd_olist_order_reviews['review_answer_timestamp'].astype('datetime64[ns]')


Visualizando informações sobre os dados

In [20]:
rwzd_olist_order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   review_id                100000 non-null  object        
 1   order_id                 100000 non-null  object        
 2   review_score             100000 non-null  UInt8         
 3   review_comment_title     11715 non-null   object        
 4   review_comment_message   41753 non-null   object        
 5   review_creation_date     100000 non-null  datetime64[ns]
 6   review_answer_timestamp  100000 non-null  datetime64[ns]
dtypes: UInt8(1), datetime64[ns](2), object(4)
memory usage: 4.8+ MB


In [22]:
rwzd_olist_order_reviews.tail()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
99995,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09,2017-12-11 20:06:42
99996,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22,2018-03-23 09:10:43
99997,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,NaN,2018-07-01,2018-07-02 12:59:13
99998,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,1,NaN,Solicitei a compra de uma capa de retrovisor c...,2017-12-15,2017-12-16 01:29:43
99999,efe49f1d6f951dd88b51e6ccd4cc548f,90531360ecb1eec2a1fbb265a0db0508,1,NaN,"meu produto chegou e ja tenho que devolver, po...",2017-07-03,2017-07-03 21:01:49


Salvando dados na camada raw em formato parquet

In [23]:
rwzd_olist_order_reviews.to_parquet(
    path='../dataset/raw/rwzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
    compression='snappy',
    index=False,
)

# Camada trusted
## Aplicando tratamentos nos dados

Importando bibliotecas necessárias

In [28]:
import pandas as pd

In [90]:
pd.set_option('display.max_colwidth', None)

Lendo dados da camada raw

In [29]:
trzd_olist_order_reviews = pd.read_parquet(
    path='../dataset/raw/rwzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
)

Filtrando por avaliações que tenham ao menos uma mensagem, mas não necessariamente o título

In [42]:
trzd_olist_order_reviews.dropna(thresh=6, inplace=True)

In [116]:
trzd_olist_order_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,2018-03-01,2018-03-02 10:26:53
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho,2018-05-22,2018-05-23 16:45:47
12,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,None,"Mas um pouco ,travando...pelo valor ta Boa.\n",2018-02-16,2018-02-20 10:52:22
16,9314d6f9799f5bfba510cc7bcd468c01,0dacf04c5ad59fd5a0cc1faa07c34e39,2,None,"GOSTARIA DE SABER O QUE HOUVE, SEMPRE RECEBI E ESSA COMPRA AGORA ME DECPCIONOU",2018-01-18,2018-01-20 21:25:45


Filtrando avaliações que comentaram sobre entregas nos comentários

In [75]:
deliver_message_mask = trzd_olist_order_reviews['review_comment_message'].str.contains(
    pat='entrega',
    case=False,
    regex=False,
    na=False,
)

In [76]:
deliver_message_mask.value_counts()

review_comment_message
False    36130
True      7352
Name: count, dtype: int64

Filtrando avaliações que comentaram sobre entregas no título

In [77]:
deliver_title_mask = trzd_olist_order_reviews['review_comment_title'].str.contains(
    pat='entrega',
    case=False,
    regex=False,
    na=False,
)

In [78]:
deliver_title_mask.value_counts()

review_comment_title
False    42915
True       567
Name: count, dtype: int64

Criando máscara para identificar avaliações que comentaram sobre entregas tanto no título quanto nos comentários

In [88]:
deliver_mask = (deliver_message_mask | deliver_title_mask)
deliver_mask.value_counts()

False    35759
True      7723
Name: count, dtype: int64

In [91]:
trzd_olist_order_reviews.loc[deliver_mask, ['review_comment_title', 'review_comment_message']].head()

,review_comment_title,review_comment_message
15,Super recomendo,"Vendedor confiável, produto ok e entrega antes do prazo."
27,None,"A compra foi realizada facilmente.\nA entrega foi efetuada muito antes do prazo dado.\nO produto já começou a ser usado e até o presente,\nsem problemas."
32,None,"Sempre compro pela Internet e a entrega ocorre antes do prazo combinado, que acredito ser o prazo máximo. No stark o prazo máximo já se esgotou e ainda não recebi o produto."
43,Entrega perfeita,Muito bom. muito cheiroso.
61,None,Ocorreu tudo como contratado sendo a entrega realizada antes do prazo \n Estou satisfeita\n


Removendo avaliações sobre entregas uma vez que o foco está nas avaliações de produtos

In [109]:
deliver_reviews_index = trzd_olist_order_reviews.loc[deliver_mask].index

In [112]:
trzd_olist_order_reviews.drop(
    index=deliver_reviews_index,
    inplace=True
)

Salvando dados na camada trusted em formato parquet

In [117]:
trzd_olist_order_reviews.to_parquet(
    path='../dataset/trusted/trzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
    compression='snappy',
    index=False,
)

# Camada delivery
## Adaptando dados para o consumo do modelo de Machine Learning

Importando bibliotecas necessárias

In [13]:
from gensim.utils import simple_preprocess

import pandas as pd

In [14]:
pd.set_option('display.max_colwidth', None)

Lendo dados da camada trusted

In [23]:
dlzd_olist_order_reviews = pd.read_parquet(
    path='../dataset/trusted/trzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
)

Utilizando o pacote *simple_preprocess*, responsável por gerar uma lista de tokens

Tranformações aplicadas:
*   Padronização em lower case
*   Desconsideração de palavras com 1 caracter e mais de 15 caracteres
*   Desconsideração de espaços em branco, números e caracteres especiais


Doc: https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html

In [24]:
dlzd_olist_order_reviews['review_comment_title'] = dlzd_olist_order_reviews['review_comment_title'].apply(
    lambda sentence: simple_preprocess(sentence, deacc=True) if sentence else []
)

dlzd_olist_order_reviews['review_comment_message'] = dlzd_olist_order_reviews['review_comment_message'].apply(
    lambda sentence: simple_preprocess(sentence, deacc=True) if sentence else []
)

Unindo título com comentários

In [26]:
titles = dlzd_olist_order_reviews['review_comment_title']
messages = dlzd_olist_order_reviews['review_comment_message']

dlzd_olist_order_reviews['review_comment_title_and_message'] = titles + messages

Reorganizando ordem das colunas

In [31]:
dlzd_olist_order_reviews = dlzd_olist_order_reviews.loc[:,
    [
        'review_id',
        'order_id',
        'review_score',
        'review_comment_title',
        'review_comment_message',
        'review_comment_title_and_message',
        'review_creation_date',
        'review_answer_timestamp'
    ]
]

Visualizando dataset transformado

In [34]:
dlzd_olist_order_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_comment_title_and_message,review_creation_date,review_answer_timestamp
0,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,[],"[recebi, bem, antes, do, prazo, estipulado]","[recebi, bem, antes, do, prazo, estipulado]",2017-04-21,2017-04-21 22:02:06
1,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,[],"[parabens, lojas, lannister, adorei, comprar, pela, internet, seguro, pratico, parabens, todos, feliz, pascoa]","[parabens, lojas, lannister, adorei, comprar, pela, internet, seguro, pratico, parabens, todos, feliz, pascoa]",2018-03-01,2018-03-02 10:26:53
2,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,[recomendo],"[aparelho, eficiente, no, site, marca, do, aparelho, esta, impresso, como, desinfector, ao, chegar, esta, com, outro, nome, atualizar, com, marca, correta, uma, vez, que, mesmo, aparelho]","[recomendo, aparelho, eficiente, no, site, marca, do, aparelho, esta, impresso, como, desinfector, ao, chegar, esta, com, outro, nome, atualizar, com, marca, correta, uma, vez, que, mesmo, aparelho]",2018-05-22,2018-05-23 16:45:47
3,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,[],"[mas, um, pouco, travando, pelo, valor, ta, boa]","[mas, um, pouco, travando, pelo, valor, ta, boa]",2018-02-16,2018-02-20 10:52:22
4,9314d6f9799f5bfba510cc7bcd468c01,0dacf04c5ad59fd5a0cc1faa07c34e39,2,[],"[gostaria, de, saber, que, houve, sempre, recebi, essa, compra, agora, me, decpcionou]","[gostaria, de, saber, que, houve, sempre, recebi, essa, compra, agora, me, decpcionou]",2018-01-18,2018-01-20 21:25:45


Salvando dados na camada delivery em formato parquet

In [35]:
dlzd_olist_order_reviews.to_parquet(
    path='../dataset/delivery/dlzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
    compression='snappy',
    index=False,
)